In [ ]:
# ! pip install langchain langchainhub langchain_openai portkey-ai

In [1]:
from dotenv import load_dotenv
import os

load_dotenv()

OPENAI_API_KEY = os.environ["OPENAI_API_KEY"]
PORTKEY_API_KEY = os.environ["PORTKEY_API_KEY"]

In [2]:
from langchain import hub  
from langchain.agents import AgentExecutor, create_openai_tools_agent  
from langchain.agents import tool
from langchain_openai import ChatOpenAI
from portkey_ai import PORTKEY_GATEWAY_URL, createHeaders

import uuid

In [3]:
portkey_headers = createHeaders(
    api_key=PORTKEY_API_KEY,
    trace_id=uuid.uuid4(),
    provider="openai",
    config="pc-basic-b390c9"
)

In [4]:
@tool  
def add(first_int: int, second_int: int) -> int:  
    "Add two integers."  
    return first_int + second_int  
  
  
@tool  
def exponentiate(base: int, exponent: int) -> int:  
    "Exponentiate the base to the exponent power."  
    return base**exponent  

@tool
def multiply(first_int: int, second_int: int) -> int:
    "Multiplies two integers"
    return first_int * second_int
  
tools = [multiply, add, exponentiate]

In [5]:
model = ChatOpenAI(api_key=OPENAI_API_KEY, base_url=PORTKEY_GATEWAY_URL, default_headers=portkey_headers, temperature=0)

In [6]:
prompt = hub.pull("hwchase17/openai-tools-agent")
# Construct the OpenAI Tools agent  
agent = create_openai_tools_agent(model, tools, prompt)

In [7]:
# Create an agent executor by passing in the agent and tools
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

In [9]:
agent_executor.invoke({
    "input": "What is 2 to the 3rd power multipied by the sum of 11 and 8"
})



> Entering new AgentExecutor chain...

Invoking: `exponentiate` with `{'base': 2, 'exponent': 3}`


8
Invoking: `add` with `{'first_int': 11, 'second_int': 8}`


19
Invoking: `multiply` with `{'first_int': 8, 'second_int': 19}`


152The result of \(2^3\) multiplied by the sum of 11 and 8 is 152.

> Finished chain.


{'input': 'What is 2 to the 3rd power multipied by the sum of 11 and 8',
 'output': 'The result of \\(2^3\\) multiplied by the sum of 11 and 8 is 152.'}